In [1]:
from math import log2
from sage.misc.sage_timeit import sage_timeit
from cryptographic_estimators.SDEstimator import SDEstimator 

def sgbp_lower_bound(n, k):
    if 2**k >= n:
        return 2**k + n
    else:
        return 2**k * 2**(n/(2**k))
        
def lgbp_lower_bound(n = 288, k = 2, pow2 = True):
    if pow2:
        K = 2**k
    else:
        K = k
    if K >= 2**20:
        st = K + n//log(K,2)
        while binomial(st, st - K) < 2**n:
            st += 1
        return st
    st = n//K
    while binomial(2**st, K) < 2**n:
        st += 1
    if binomial(2**st, K) == 2**n:
        return 2**st
    # binary search
    ub = 2**(st)
    lb = 2**(st - 1)
    while lb + 1 < ub:
        mid = (ub + lb) // 2
        total = binomial(mid, K)
        if total < 2**n:
            lb = mid
        elif total > 2**n:
            ub = mid
        else:
            return mid
    assert binomial(lb, K) < 2**n
    assert binomial(ub, K) >= 2**n
    return ub
    
def k_list_time_estimator(n, k):
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    # the time complexity of Wagner's algorithm should be 2^{ell + k + 1} instead of 2^{ell + k}
    # 2^{k- 1} * 2^{ell + 1} hases
    # (2^{k}- 1) * 2^{ell + 1} time for merge
    return (2^k + 2^(k-1) -  1) * 2^(1 + ell)

def k_list_mem_estimator(n, k):
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    N = 2^(ell + 1)
    return ((k**2 + 5*k + 2)/4 + 2**(k-1)) * ell * N
    
def k_list_mem_estimator_star(n, k):
    # Wagner's algorithm for SGBP(n, K=2^k)
    # ell = ceil(n/(k + 1))
    ell = n/(k + 1)
    N = 2^(ell + 1)
    return ((k**2 + k - 6)/4 + 2**k) * ell * N

def single_list_time_estimator(n, k):
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    return k * N

def single_list_mem_estimator_star(n, k):
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    return 2*(n + k - ell - 1) * N

def single_list_mem_estimator_plain(n, k):
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    return (2**(k-1) *(ell + 1)+ 2*ell) * N

def single_list_mem_estimator_equihash(n, k):
    # optimized version mentioned in Equihash, stores only 8 bits index in the last round.
    # the trade-off should be reconsidered
    # ell = ceil(n /(k+1))
    ell = n/(k+1)
    N = 2^(ell + 1)
    # (2**(k-1) *(ell + 1)+ 2*ell) * N
    return 2**(ell + 3) * (2**k + ell/2)
    
def single_list_k_upper_bound(n):
    return sqrt(n/2 + 1)

def to_log2_complexity(complexity):
    """
    Convert the complexity to log2 scale.
    """
    return RR(log(complexity, 2))

def to_MB(mem_complexity):
    """
    Convert the memory complexity (bits) to MB.
    """
    return RR(mem_complexity) / (8 * 1024 * 1024)

def to_GB(mem_complexity):
    """
    Convert the memory complexity (bits) to GB.
    """
    return RR(mem_complexity) / (8 * 1024 * 1024 * 1024)

def to_TB(mem_complexity):
    """
    Convert the memory complexity (bits) to TB.
    """
    return RR(mem_complexity) / (8 * 1024 * 1024 * 1024 * 1024)

def k_list_reducded_size(n, k, t, using_trade_off=False):
    """ K-list algorithm with list size reduced to N / 2^t.
    """
    ell = n/(k + 1)
    # size of the list in height h
    if using_trade_off: 
        idx_len = 1
    else: 
        idx_len = ell - t
    N_h = lambda h: 2 ** (ell - t * 2**h)
    M = (ell + n) * N_h(0)
    T = 0
    for i in range(0, k):
        M += ((2**i) * idx_len  + n - i * ell) * N_h(i)
        T += 2**(k - 1 - i) * 2 * N_h(i)
    return M, T

def best_k_list_memory_trade_off(n, k):
    """
    The index trimming technique can be applied in K-list algorithm while not increasing the time complexity significantly. 
    We search the best index trimming bit length for K-list algorithm. We only consider the depth of 2.
    """
    ell = n/(k + 1)
    N = 2^(ell + 1)
    M_xor = ((k**2 + 5*k + 2)/4) * ell * N
    M_idx_t = lambda t: 2**(k-1) * t * N
    M_first_run = lambda t: M_xor + M_idx_t(t)
    # N_t = lambda t: N / 2**t
    # M_second_run = lambda t: ((k**2 + 5*k + 2)/4) * ell * N_t(t) +  2**(k-1) * ell * N_t(t)
    for t in range(1, ceil(ell)):
        first_run_mem = M_first_run(t)
        second_run_mem, _ = k_list_reducded_size(n, k, t)
        if first_run_mem >= second_run_mem:
            return t, first_run_mem

def best_single_list_memory_trade_off_depth2(n, k):
    """
    The index trimming technique can be applied in single-list algorithm while not increasing the time complexity significantly. 
    We will limit the XOR-removal technique below height floor(log(k,2)) such that the time penalty is within twofold.
    We search the best index trimming bit length for single-list algorithm with depth of 2.
    """
    ell = n/(k + 1)
    N = 2^(ell + 1)
    T0 = k * N
    xr_height = floor(log(k,2))
    def M_t(t):
        if 2**(k-1) * t  + 2 * ell > n:
            return (2**(k-1) * t  + 2 * ell) * N, 0
        xor_removal_mems = [max(2**height * (ell + 1), n - (height + 1) * ell + 2**(height + 1) * t) for height in range(1, xr_height + 1)]
        xor_removal_best_mem_t = min(xor_removal_mems)
        switching_height = xor_removal_mems.index(xor_removal_best_mem_t) + 1
        M = max(2**(k-1) * t  + 2 * ell, xor_removal_best_mem_t) * N
        return M, switching_height

    best_m = +infinity
    for t in range(1, ceil(ell)):
        first_run_mem, switching_height =  M_t(t)
        second_run_mem = single_list_with_constraint(n, k, t, use_xr=1)
        mem = max(first_run_mem, second_run_mem)
        if mem < best_m:
            best_m = mem
            best_case_is_first_run = first_run_mem > second_run_mem
            best_switching_height = switching_height
            best_t = t
    print(f"{(n,k) = }, best trade-off for depth-2 with limited XOR-removal: {best_t = }, {best_switching_height = }")
    return best_t, best_m

def single_list_with_constraint(n, k, t, use_xr = 1):
    """
    estimate the peak memory of single-list algorithm with partial t-bit solution vector constraint.
    use_xr = -1, 0, 1 ==> unlimited xor-removal, no xor-removal, limited xor-removal
    """
    # max_candi_length = lambda h: 2**(k - h)
    max_permutations = [2**t]
    max_candidates = [2**k]
    t_h = None
    ell = n/(k + 1)
    N = 2^(ell + 1)
    find = False
    for i in range(1, k):
        max_cand_i = 2**(k - i)
        max_perm_i = (2**t) ** (2**i)
        max_candidates.append(max_cand_i)
        max_permutations.append(max_perm_i)
        if max_cand_i < max_perm_i and not find:
            # the constraint will be activated at height i
            t_h = i - 1 # the list at t_h is of size O(N) and the subsequent list size will be << N.
            find = True
            # print(f"Trade-off for single-list algorithm: {n = } {k = } {t = } {t_h = }")
            # break
    def xor_removal_use_case(h):
        M1 = 2**h * (ell + 1) * N
        if use_xr == 1:
            # use xor removal in limited case
            can_xor_removal = 2**(t_h) <= k
            if not can_xor_removal:
                M1 += (n - h * ell) * N
        elif use_xr == 0:
            # do not use xor removal
             M1 += (n - h * ell) * N
        elif use_xr == -1:
            # always use xor removal
            return M1
    # memory complexity before the list size significantly reduced        
    M1 = xor_removal_use_case(t_h)
    # the reduced_size at height t_h + 1
    reduced_size_N = (max_candidates[t_h + 1] / max_permutations[t_h + 1]) * N
    M2 =  2**(t_h + 1) * (ell + 1) * reduced_size_N +  (n - (t_h + 1) * ell) * reduced_size_N
    M = max(M2, M1)
    # try next height if the reduced_size_N is not too small
    if t_h + 2 >= k - 1:
        return M
    t_h += 1
    M1 = xor_removal_use_case(t_h)
    reduced_size_N = (max_candidates[t_h + 1] / max_permutations[t_h + 1]) * binomial(reduced_size_N, 2) / 2**ell  
    M2 =  2**(t_h + 1) * (ell + 1) * reduced_size_N +  (n - (t_h + 1) * ell) * reduced_size_N
    M_prime = max(M2, M1)
    return min(M, M_prime)
    
def equihash_solution_size(n, k, nonce_bits = 160):
    assert n % (k+1) == 0, "Invalid parameters"
    ell = n/(k+1)
    return (2**k) * (ell + 1) + nonce_bits

def sequihash_solution_size(n, k, nonce_bits = 160):
    assert n % (k+1) == 0, "Invalid parameters"
    ell = n/(k+1)
    # the index bit can be omitted in network transmission.
    # return (2**k) * (k + ell) + nonce_bits
    return (2**k) * ell + nonce_bits

def optimal_single_list_mem(n, k):
    ell = n/(k + 1)
    N = 2^(ell + 1)
    T0 = k * N
    return (2**(k-1)  + 2 * ell) * N

def optimal_k_list_mem(n, k):
    ell = n/(k + 1)
    N = 2^(ell + 1)
    M_xor = ((k**2 + 5*k + 2)/4) * ell * N
    M_idx_t = lambda t: 2**(k-1) * t * N
    return M_xor + M_idx_t(1)

In [2]:
Equihash_Parameter_Set = [
    (96, 5),
    (128, 7),
    (160, 9),
    (96, 3),
    (144, 5),
    (150, 5),
    (200, 9),
    (288, 8)
]

nonce_bits = 0 # this is 160 in Equihash. We set it as zero for the clarity of comparisons
# WE USE TRADE-OFF HERE, doubles the complexity
T_lgbp = [log(single_list_time_estimator(n, k), 2) + 1 for n, k in Equihash_Parameter_Set]
T_sgbp =  [log(k_list_time_estimator(n, k), 2) + 2 for n, k in Equihash_Parameter_Set]
single_list_trade_offs = [best_single_list_memory_trade_off_depth2(n, k) for n, k in Equihash_Parameter_Set]
M_lgbp = [to_log2_complexity(mem) for t, mem in single_list_trade_offs]
k_list_trade_offs = [best_k_list_memory_trade_off(n, k) for n, k in Equihash_Parameter_Set]
M_sgbp = [to_log2_complexity(mem) for t, mem in k_list_trade_offs]
Sz_lgbp = [ceil(equihash_solution_size(n,k, nonce_bits)/8) for n, k in Equihash_Parameter_Set]
Sz_sgbp = [ceil(sequihash_solution_size(n, k, nonce_bits)/8) for n, k in Equihash_Parameter_Set]

gaplen = 9
print(f"%{gaplen}s %{gaplen*3}s %{gaplen*3}s" % ("(n, k)", "Equihash", "Sequihash"))
print(f"%{gaplen}s %{gaplen}s %{gaplen}s %{gaplen}s %{gaplen}s %{gaplen}s %{gaplen}s" % 
      ("    ", "Time", "Mem.", "Size", "Time", "Mem.", "Size"))

for i, (n,k) in enumerate(Equihash_Parameter_Set):
    print(f"%{gaplen}s %{gaplen}.1f %{gaplen}.1f %{gaplen}sB %{gaplen}.1f %{gaplen}.1f %{gaplen}sB" % 
          ((n, k), RR(T_lgbp[i]), RR(M_lgbp[i]), Sz_lgbp[i], RR(T_sgbp[i]), RR(M_sgbp[i]), Sz_sgbp[i]))


(n,k) = (96, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (128, 7), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 2
(n,k) = (160, 9), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (96, 3), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (144, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (150, 5), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 1
(n,k) = (200, 9), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 0
(n,k) = (288, 8), best trade-off for depth-2 with limited XOR-removal: best_t = 1, best_switching_height = 2
   (n, k)                    Equihash                   Sequihash
               Time      Mem.      Size      Time      Mem.     

In [3]:
for i, (n,k) in enumerate(Equihash_Parameter_Set):
    print(f"({n}, $2^{{{k}}}$) &  $2^{{{RR(T_lgbp[i]):.1f}}}$ & $2^{{{RR(M_lgbp[i]):.1f}}}$ & {Sz_lgbp[i]} B", end=" & ")
    print(f"$2^{{{RR(T_sgbp[i]):.1f}}}$ & $2^{{{RR(M_sgbp[i]):.1f}}}$ & {Sz_sgbp[i]} B", end="\\\\")
    print()

(96, $2^{5}$) &  $2^{20.3}$ & $2^{23.1}$ & 68 B & $2^{24.6}$ & $2^{24.8}$ & 64 B\\
(128, $2^{7}$) &  $2^{20.8}$ & $2^{23.6}$ & 272 B & $2^{26.6}$ & $2^{25.7}$ & 256 B\\
(160, $2^{9}$) &  $2^{21.2}$ & $2^{25.2}$ & 1088 B & $2^{28.6}$ & $2^{26.6}$ & 1024 B\\
(96, $2^{3}$) &  $2^{27.6}$ & $2^{30.7}$ & 25 B & $2^{30.5}$ & $2^{32.3}$ & 24 B\\
(144, $2^{5}$) &  $2^{28.3}$ & $2^{31.6}$ & 100 B & $2^{32.6}$ & $2^{33.4}$ & 96 B\\
(150, $2^{5}$) &  $2^{29.3}$ & $2^{32.7}$ & 104 B & $2^{33.6}$ & $2^{34.4}$ & 100 B\\
(200, $2^{9}$) &  $2^{25.2}$ & $2^{29.2}$ & 1344 B & $2^{32.6}$ & $2^{30.8}$ & 1280 B\\
(288, $2^{8}$) &  $2^{37.0}$ & $2^{40.6}$ & 1056 B & $2^{43.6}$ & $2^{42.9}$ & 1024 B\\


In [4]:
from prettytable import PrettyTable
table_fields = ['(n,k)',' ', 'Equihash', '   ', '  ', 'Sequihash', '    ']
pt = PrettyTable(table_fields)
gaplen = 4
pt.add_row([' ', 'Time', 'Mem.', 'Size', 'Time', 'Mem.', 'Size'], divider=True)
for i, (n,k) in enumerate(Equihash_Parameter_Set):
    pt.add_row([str((n,k)),f"%{gaplen}.1f"%RR(T_lgbp[i]), f"%{gaplen}.1f"%RR(M_lgbp[i]), f"%{gaplen}sB"% Sz_lgbp[i],
                f"%{gaplen}.1f"%RR(T_sgbp[i]), f"%{gaplen}.1f"%RR(M_sgbp[i]), f"%{gaplen}sB"% Sz_sgbp[i]])


In [5]:
print(pt)

+----------+------+----------+-------+------+-----------+-------+
|  (n,k)   |      | Equihash |       |      | Sequihash |       |
+----------+------+----------+-------+------+-----------+-------+
|          | Time |   Mem.   |  Size | Time |    Mem.   |  Size |
+----------+------+----------+-------+------+-----------+-------+
| (96, 5)  | 20.3 |   23.1   |   68B | 24.6 |    24.8   |   64B |
| (128, 7) | 20.8 |   23.6   |  272B | 26.6 |    25.7   |  256B |
| (160, 9) | 21.2 |   25.2   | 1088B | 28.6 |    26.6   | 1024B |
| (96, 3)  | 27.6 |   30.7   |   25B | 30.5 |    32.3   |   24B |
| (144, 5) | 28.3 |   31.6   |  100B | 32.6 |    33.4   |   96B |
| (150, 5) | 29.3 |   32.7   |  104B | 33.6 |    34.4   |  100B |
| (200, 9) | 25.2 |   29.2   | 1344B | 32.6 |    30.8   | 1280B |
| (288, 8) | 37.0 |   40.6   | 1056B | 43.6 |    42.9   | 1024B |
+----------+------+----------+-------+------+-----------+-------+
